In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sorular.csv to sorular.csv


In [ ]:
import pandas as pd

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

,ID,Soru_Metni(OCR),Alt_Konu,Zorluk,Görsel_Tipi,Kaynak_Tipi
0,P2-S1,18 sayısını bölen pozitif tam sayıların toplam...,Bir Doğal Sayının Çarpanları,1,Görsel Yok,Başlangıç Sorusu
1,P2-S2,Aşağıdakilerden hangisi 96 sayısının çarpanlar...,Bir Doğal Sayının Çarpanları,1,Görsel Yok,Başlangıç Sorusu
2,P2-S3,Aşağıdakilerden hangisi bu dikdörtgensel bölge...,Bir Doğal Sayının Çarpanları,2,Görsel Yok,Başlangıç Sorusu
3,P2-S4,Bu dikdörtgenlerden çevresi en küçük olanın çe...,Bir Doğal Sayının Çarpanları,3,Görsel Yok,Başlangıç Sorusu
4,P2-S5,Misafirlerin sayısı 6'dan fazla olduğuna göre ...,Bir Doğal Sayının Çarpanları,2,Görsel Yok,Başlangıç Sorusu


In [ ]:
df.columns

Index(['ID', 'Soru_Metni(OCR)', 'Alt_Konu', 'Zorluk', 'Görsel_Tipi',
       'Kaynak_Tipi'],
      dtype='object')

In [ ]:
def clean_column_name(col):
    return (col
            .strip()
            .replace(" ", "_")
            .replace("-", "_")
            .replace("(", "")
            .replace(")", "")
            .replace("Ö", "O")
            .replace("ö", "o")
            .replace("Ü", "U")
            .replace("ü", "u")
            .replace("Ş", "S")
            .replace("ş", "s")
            .replace("Ğ", "G")
            .replace("ğ", "g")
            .replace("İ", "I")
            .replace("ı", "i")
           )
df.columns = [clean_column_name(c) for c in df.columns]
df.columns


Index(['ID', 'Soru_MetniOCR', 'Alt_Konu', 'Zorluk', 'Gorsel_Tipi',
       'Kaynak_Tipi'],
      dtype='object')

In [ ]:
df['Zorluk'] = df['Zorluk'].astype(int)

In [ ]:
df['Zorluk'] = df['Zorluk'].astype(str).str.strip().astype(int)

In [ ]:
df['Gorsel_Tipi'].unique()

array(['Görsel Yok', 'Resimli', 'Geometrik Şekil', 'Şematik', 'Yok',
       'Tablo'], dtype=object)

In [ ]:
# Normalize Görsel Tipi
def normalize_gorsel(x):
    x = str(x).strip()
    mapping = {
        "Görsel Yok": "yok",
        "Yok": "yok",
        "Resimli": "resimli",
        "Geometrik Şekil": "geometrik_sekil",
        "Şematik": "sematik",
        "Tablo": "tablo",
    }
    return mapping.get(x, x.lower().replace(" ", "_"))

df["Gorsel_Tipi"] = df["Gorsel_Tipi"].apply(normalize_gorsel)

df["Gorsel_Tipi"].unique()


array(['yok', 'resimli', 'geometrik_sekil', 'sematik', 'tablo'],
      dtype=object)

In [ ]:
df['Kaynak_Tipi'].unique()

array(['Başlangıç Sorusu', 'LGS Örnek Sorusu', 'LGS Örnek Soru',
       'Çıkmış Soru (2025)', 'Çıkmış Soru (2024)', 'Çıkmış Soru (2023)',
       'Çıkmış Soru (2022)', 'Çıkmış Soru (2021)', 'Çıkmış Soru (2020)',
       'Çıkmış Soru (2019)', 'Çıkmış Soru (2018)'], dtype=object)

In [ ]:
def normalize_kaynak(x):
    x = str(x).strip()

    if "Başlangıç" in x:
        return "baslangic"

    if "Örnek" in x or "Ornek" in x:
        return "ornek"

    if "Çıkmış" in x or "Cikmis" in x:
        return "cikmis"

    return x.lower()

df["Kaynak_Tipi"] = df["Kaynak_Tipi"].apply(normalize_kaynak)
df["Kaynak_Tipi"].unique()


array(['baslangic', 'ornek', 'cikmis'], dtype=object)

In [ ]:
df['Alt_Konu'].unique()

array(['Bir Doğal Sayının Çarpanları', 'EBOB ve EKOK',
       'Aralarında Asal Sayılar'], dtype=object)

In [ ]:
df['Alt_Konu'] = df['Alt_Konu'].str.strip()

In [ ]:
df['Alt_Konu'].unique()

array(['Bir Doğal Sayının Çarpanları', 'EBOB ve EKOK',
       'Aralarında Asal Sayılar'], dtype=object)

In [ ]:
alt_konu_map = {
    "Bir Doğal Sayının Çarpanları": "carpanlar",
    "EBOB ve EKOK": "ebob_ekok",
    "Aralarında Asal Sayılar": "aralarinda_asal",
}

df["Alt_Konu"] = df["Alt_Konu"].map(alt_konu_map)
df["Alt_Konu"].unique()


array(['carpanlar', 'ebob_ekok', 'aralarinda_asal'], dtype=object)

In [ ]:
def get_weight(x):
    return {
        "cikmis": 1.0,
        "ornek": 0.8,
        "baslangic": 0.3
    }.get(x, 1.0)

df["sample_weight"] = df["Kaynak_Tipi"].apply(get_weight)

In [ ]:
df.head()

,ID,Soru_MetniOCR,Alt_Konu,Zorluk,Gorsel_Tipi,Kaynak_Tipi,sample_weight
0,P2-S1,18 sayısını bölen pozitif tam sayıların toplam...,carpanlar,1,yok,baslangic,0.3
1,P2-S2,Aşağıdakilerden hangisi 96 sayısının çarpanlar...,carpanlar,1,yok,baslangic,0.3
2,P2-S3,Aşağıdakilerden hangisi bu dikdörtgensel bölge...,carpanlar,2,yok,baslangic,0.3
3,P2-S4,Bu dikdörtgenlerden çevresi en küçük olanın çe...,carpanlar,3,yok,baslangic,0.3
4,P2-S5,Misafirlerin sayısı 6'dan fazla olduğuna göre ...,carpanlar,2,yok,baslangic,0.3


In [ ]:
df = df.drop(columns=["ID"])

In [ ]:
df.to_csv("cleaned_dataset.csv", index=False)

In [ ]:
df["is_LGS"] = (df["Kaynak_Tipi"].isin(["ornek", "cikmis"])).astype(int)

In [ ]:
# Kolon adı
col = "Soru_MetniOCR"

# 1) Kelime sayisi
df["ocr_kelime_sayisi"] = df[col].str.split().str.len()

# 2) Karakter sayisi
df["ocr_karakter_sayisi"] = df[col].str.len()

# 3) Rakam sayisi
df["ocr_rakam_sayisi"] = df[col].str.count(r"\d")

# 4) Cok adimli muhakeme ifadesi
reasoning_words = [
    "en az", "en cok", "en fazla",
    "buna gore", "once", "daha sonra",
    "en son", "ardindan", "son durumda"
]

pattern_reason = "|".join(reasoning_words)
df["ocr_cok_adimli"] = df[col].str.contains(
    pattern_reason, case=False, regex=True
).astype(int)


In [ ]:
df = df.rename(columns={"sample_weight": "egitim_agirligi"})

In [ ]:
df.to_csv("dataset_ocr_li.csv", index=False, encoding="utf-8")


In [ ]:
df_no_ocr = df.drop(columns=["Soru_MetniOCR"])


In [ ]:
df_no_ocr.to_csv("dataset_model_final.csv", index=False, encoding="utf-8")


In [ ]:
df.head()

,Soru_MetniOCR,Alt_Konu,Zorluk,Gorsel_Tipi,Kaynak_Tipi,egitim_agirligi,is_LGS,ocr_kelime_sayisi,ocr_karakter_sayisi,ocr_rakam_sayisi,ocr_cok_adimli
0,18 sayısını bölen pozitif tam sayıların toplam...,carpanlar,1,yok,baslangic,0.3,0,8,55,2,0
1,Aşağıdakilerden hangisi 96 sayısının çarpanlar...,carpanlar,1,yok,baslangic,0.3,0,7,66,2,0
2,Aşağıdakilerden hangisi bu dikdörtgensel bölge...,carpanlar,2,yok,baslangic,0.3,0,9,84,0,0
3,Bu dikdörtgenlerden çevresi en küçük olanın çe...,carpanlar,3,yok,baslangic,0.3,0,10,63,0,0
4,Misafirlerin sayısı 6'dan fazla olduğuna göre ...,carpanlar,2,yok,baslangic,0.3,0,12,95,1,0


In [ ]:
# 1) Kolon adını değiştir
df = df.rename(columns={"Soru_MetniOCR": "soru_metni"})

# Kontrol
print("soru_metni" in df.columns, df.columns.tolist())

# 2) İstersen dosyaya kaydet (RAG için)
df.to_csv("dataset_rag_ready.csv", index=False, encoding="utf-8-sig")

print("Kaydedildi: dataset_rag_ready.csv")


True ['soru_metni', 'Alt_Konu', 'Zorluk', 'Gorsel_Tipi', 'Kaynak_Tipi', 'egitim_agirligi', 'is_LGS', 'ocr_kelime_sayisi', 'ocr_karakter_sayisi', 'ocr_rakam_sayisi', 'ocr_cok_adimli']
Kaydedildi: dataset_rag_ready.csv


In [ ]:
# === MODEL HAZIRLIK KONTROLÜ ===

print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

# 1) Eksik değer kontrolü
na = df.isna().sum()
print("\nEksik degerler:")
print(na[na > 0] if (na > 0).any() else "YOK ✅")

# 2) Tipler
print("\nDtypes:")
print(df.dtypes)

# 3) Unique değerler
for col in ["Alt_Konu", "Zorluk", "Gorsel_Tipi", "Kaynak_Tipi", "is_LGS"]:
    print(f"\n{col} unique:", sorted(df[col].unique()))

# 4) Sınıf dağılımı
print("\nis_LGS dagilimi:")
print(df["is_LGS"].value_counts())

print("\nMODEL ICIN HAZIRLIK KONTROLU BITTI ✅")


Shape: (176, 11)

Columns: ['soru_metni', 'Alt_Konu', 'Zorluk', 'Gorsel_Tipi', 'Kaynak_Tipi', 'egitim_agirligi', 'is_LGS', 'ocr_kelime_sayisi', 'ocr_karakter_sayisi', 'ocr_rakam_sayisi', 'ocr_cok_adimli']

Eksik degerler:
YOK ✅

Dtypes:
soru_metni              object
Alt_Konu                object
Zorluk                   int64
Gorsel_Tipi             object
Kaynak_Tipi             object
egitim_agirligi        float64
is_LGS                   int64
ocr_kelime_sayisi        int64
ocr_karakter_sayisi      int64
ocr_rakam_sayisi         int64
ocr_cok_adimli           int64
dtype: object

Alt_Konu unique: ['aralarinda_asal', 'carpanlar', 'ebob_ekok']

Zorluk unique: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]

Gorsel_Tipi unique: ['geometrik_sekil', 'resimli', 'sematik', 'tablo', 'yok']

Kaynak_Tipi unique: ['baslangic', 'cikmis', 'ornek']

is_LGS unique: [np.int64(0), np.int64(1)]

is_LGS dagilimi:
is_LGS
1    89
0    87
Name: count, dtype: int64

MODEL ICIN HAZIRLI

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# --------------------
# Feature / target
# --------------------
X = df[
    [
        "Alt_Konu",
        "Zorluk",
        "Gorsel_Tipi",
        "ocr_kelime_sayisi",
        "ocr_karakter_sayisi",
        "ocr_rakam_sayisi",
        "ocr_cok_adimli",
    ]
]

y = df["is_LGS"]
sample_weight = df["egitim_agirligi"]

cat_cols = ["Alt_Konu", "Gorsel_Tipi"]
num_cols = [
    "Zorluk",
    "ocr_kelime_sayisi",
    "ocr_karakter_sayisi",
    "ocr_rakam_sayisi",
    "ocr_cok_adimli",
]

# --------------------
# Preprocess
# --------------------
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# --------------------
# Model
# --------------------
model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("clf", LogisticRegression(max_iter=1000)),
    ]
)

# --------------------
# Train / test split
# --------------------
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X,
    y,
    sample_weight,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

# --------------------
# Fit
# --------------------
model.fit(X_train, y_train, clf__sample_weight=w_train)

# --------------------
# Quick sanity metric
# --------------------
y_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_proba)

print(f"Model egitildi ✅ | ROC-AUC: {auc:.3f}")


Model egitildi ✅ | ROC-AUC: 0.960


In [ ]:
import json
import numpy as np
import pandas as pd
from datetime import datetime

# -----------------------------
# 1) Kombinasyon uzayı (tüm olası kombinasyonlar)
# -----------------------------
alt_konu_list = sorted(df["Alt_Konu"].unique())
zorluk_list = sorted(df["Zorluk"].unique())
gorsel_list = sorted(df["Gorsel_Tipi"].unique())

combos = []
for a in alt_konu_list:
    for z in zorluk_list:
        for g in gorsel_list:
            combos.append({"Alt_Konu": a, "Zorluk": int(z), "Gorsel_Tipi": g})

combos_df = pd.DataFrame(combos)

# -----------------------------
# 2) Kombinasyonlar için "tipik OCR" değerleri
#    (RAG'e stabil skor üretmek için: median bazlı)
#    Not: Burada tüm dataset genel median'ı kullanıyoruz.
#    İstersen Alt_Konu bazlı median da yapabiliriz.
# -----------------------------
ocr_cols = ["ocr_kelime_sayisi","ocr_karakter_sayisi","ocr_rakam_sayisi","ocr_cok_adimli"]
med = df[ocr_cols].median(numeric_only=True)

for c in ocr_cols:
    combos_df[c] = int(med[c])

# -----------------------------
# 3) Model ile lgs_skor hesapla (predict_proba)
# -----------------------------
X_combo = combos_df[
    ["Alt_Konu","Zorluk","Gorsel_Tipi",
     "ocr_kelime_sayisi","ocr_karakter_sayisi","ocr_rakam_sayisi","ocr_cok_adimli"]
]

combos_df["lgs_skor"] = model.predict_proba(X_combo)[:, 1]

# -----------------------------
# 4) Coverage (RAG için kritik)
#    RAG en az 3 örnek istiyor.
#    Retrieval mantığına benzer şekilde:
#      - aynı Alt_Konu
#      - zorluk ±1
#      - kaynak: cikmis/ornek (baslangic hariç)
#      - is_LGS = 1 (öncelik)
# -----------------------------
def coverage_count(row):
    a = row["Alt_Konu"]
    z = row["Zorluk"]
    z_min, z_max = z-1, z+1

    sub = df[
        (df["Alt_Konu"] == a) &
        (df["Zorluk"].between(z_min, z_max)) &
        (df["Kaynak_Tipi"].isin(["cikmis", "ornek"])) &
        (df["is_LGS"] == 1)
    ]
    return int(len(sub))

combos_df["coverage"] = combos_df.apply(coverage_count, axis=1)

# Min 3 örnek yoksa RAG patlayabiliyor -> flag
MIN_EXAMPLES = 3
combos_df["coverage_ok"] = (combos_df["coverage"] >= MIN_EXAMPLES).astype(int)

# -----------------------------
# 5) Threshold + sıralama
#    Önce coverage_ok=1 olanları tut, sonra skora göre sırala
# -----------------------------
THRESHOLD = 0.75

filtered = combos_df[(combos_df["coverage_ok"] == 1) & (combos_df["lgs_skor"] >= THRESHOLD)].copy()
filtered = filtered.sort_values("lgs_skor", ascending=False).reset_index(drop=True)
filtered["rank"] = np.arange(1, len(filtered) + 1)

print("Toplam kombinasyon:", len(combos_df))
print("Threshold+coverage sonrası:", len(filtered))

# -----------------------------
# 6) configs.json yaz
# -----------------------------
configs = {
    "schema_version": "1.0",
    "model_version": "lgs-struct-score-1.0",
    "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "threshold": float(THRESHOLD),
    "min_examples": int(MIN_EXAMPLES),
    "total_combinations": int(len(combos_df)),
    "filtered_combinations": int(len(filtered)),
    "selection_policy": {
        "mode": "weighted_sampling",   # RAG bunu kullanabilir
        "alpha": 2.0,                  # ağırlık = skor^alpha
        "temperature": 1.0,            # softmax yumuşatma
        "top_k": min(30, int(len(filtered)))  # istersen
    },
    "combinations": [
        {
            "alt_konu": r["Alt_Konu"],
            "zorluk": int(r["Zorluk"]),
            "gorsel_tipi": r["Gorsel_Tipi"],
            "lgs_skor": float(r["lgs_skor"]),
            "coverage": int(r["coverage"]),
            "rank": int(r["rank"])
        }
        for _, r in filtered.iterrows()
    ]
}

with open("configs.json", "w", encoding="utf-8") as f:
    json.dump(configs, f, ensure_ascii=False, indent=2)

# Debug csv (tam tablo)
combos_df_sorted = combos_df.sort_values("lgs_skor", ascending=False).reset_index(drop=True)
combos_df_sorted.to_csv("combination_scores.csv", index=False, encoding="utf-8-sig")

print("✅ configs.json ve combination_scores.csv kaydedildi")
print("İlk 5 kombinasyon:")
print(filtered[["Alt_Konu","Zorluk","Gorsel_Tipi","lgs_skor","coverage"]].head())


Toplam kombinasyon: 75
Threshold+coverage sonrası: 40
✅ configs.json ve combination_scores.csv kaydedildi
İlk 5 kombinasyon:
          Alt_Konu  Zorluk      Gorsel_Tipi  lgs_skor  coverage
0  aralarinda_asal       5          resimli  0.997996         6
1  aralarinda_asal       5          sematik  0.997358         6
2        carpanlar       5          resimli  0.997009         6
3  aralarinda_asal       5  geometrik_sekil  0.996541         6
4        ebob_ekok       5          resimli  0.996126        40
